In [14]:
import numpy as np
import math
from matplotlib.pyplot import cm
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# plt.rcParams.update({'font.size': 20,'figure.figsize': (10.0, 6.0)})
import ipywidgets as wg
from ipywidgets import Layout
# %matplotlib widget
%matplotlib qt
import time
import csv
import matplotlib
from scipy.signal import savgol_filter
import mpmath
from scipy import interpolate



In [15]:
num=200

#Material Properties and global variables

h=6.62607004e-34   #Plank's Constant
kb=1.3806485e-23   #Boltzman Constant
mu0=1.257e-6       #Air Permeability
hbar=h/(2*np.pi)


#NbTiN
Tc_NbTiN=12.4            #Critical Temperature
delta_NbTiN=1.764*kb*Tc_NbTiN #Order parameter/ Energy 
Jc_NbTiN= 1.5e11        #critical current density in A/m
ro_NbTiN=242e-8             #resistivity of NbTiN
 
 #Nb
Tc_Nb=9.5            #Critical Temperature
delta_Nb=1.764*kb*Tc_Nb #Order parameter/ Energy 
Jc_Nb= 2e12       #critical current density in A/m
ro_Nb=1.5e-7             #resistivity of NbTiN
    

In [53]:
def geoindfun (l,t,w,lam): #van duzer See Figure 1 first equation 
    L=l*(mu0/(2*w))*(((lam/2)*math.sinh(2*t/lam)-t)/((math.sinh(t/lam))**2))
    return L


# def kinindfun (l,t,w,ro): #Annunziata 
#     L=(l/w)*((ro/t)*h)/(2*(np.pi**2)*delta*(math.tanh(delta/(2*kb*Tc))))
#     return L

def kinindfun (l,t,w,lam): #van duzer See Figure 1 second equation
    L=l*(mu0/(2*w))*(((lam/2)*math.sinh(2*t/lam)+t)/((math.sinh(t/lam))**2))
    return L

def lambdafun(ro,Tc): ##See figure#2 below 
    lambda_m=np.sqrt((hbar*ro)/(np.pi*delta*mu0))
    return lambda_m

def totalindfun (Lint,l,w,d): #van duzer eq11 chapter 3
    L=Lint+l*mu0*d/w
    return L

# def totalindfun (l,t1,t2,w,lam1,lam2,d): #van duzer eq11 chapter 3
    
#     #d is the spacing between the top and bottom conductors
#     #lam1 is the penetration depth of the first conductor
#     #lam2 is the penetration depth of the second conductor
#     #t1 is the thickness of the first conductor
#     #t2 is the thickness of the second conductor

#     L=l*(mu0/(K*w))*(1+(lam1/d)*mpmath.coth(t1/lam1)+(lam2/d)*mpmath.coth(t2/lam2))
#     return L

# Ls=12.7e-12
# ro=Ls/(((1/t)*h)/(2*(np.pi**2)*delta*(math.tanh(delta/(2*kb*Tc)))))

lambda_NbTiN=lambdafun(ro_NbTiN,Tc_NbTiN)
# print('ro=',ro*1e8, 'u ohm cm')
print('lambda_NbTiN=',lambda_NbTiN*1e9, 'nm')


lambda_Nb=lambdafun(ro_Nb,Tc_Nb)
# print('ro=',ro*1e8, 'u ohm cm')
print('lambda_Nb=',lambda_Nb*1e9, 'nm')


lambda_NbTiN= 462.5955908526365 nm
lambda_Nb= 115.17001823291795 nm


In [11]:
##Interpolation to the above graph of the slow esperiments model 

w = []
x = []
y = []
    
with open('ClemData.csv', newline='') as f:
    reader = csv.reader(f, delimiter=' ')
    
    for row in reader:
            w.extend(row)
    
           
for i in range(len(w)):
    if i%2==1:
        y.append(float(w[i]))
    else:
        x.append(float(w[i]))
        

z=[0.3,0.5,0.9]
tun= 1/np.interp(z, x, y)
print(tun)


[1.03778407 1.1215706  2.03705961]


In [135]:
#Ratios Comparison (Many)
lambda_NbTiN=lambdafun(ro,Tc)#NbTiN Penetration Depth 

t1= 20e-9 #thickness of the first inductor 
w1=(100/30)*0.25e-6; #width of the first inductor 
ar1=w1/t1; #aspect ratio of the first inductor 
l1= 1.1e-6 #length of the first inductor 
Ic1= Jc*t1*w1 #critical current of the first inductor
Lm1= geoindfun(l1,t1,w1,lam=lambda_NbTiN) #Calculating magnetic inductance for the first inductor
Lk1= kinindfun(l1,t1,w1,lam=lambda_NbTiN) #Calculating kinetic inductance for the first inductor
Ls1=Lk1*w1/l1 #sheet inductance for the first inductor
t2= 20e-9

n=int(1) #Number of different ratios you want to compare 

color = iter(cm.rainbow(np.linspace(0, 1, n)))

for i in range(n):
    
    w2=(200/(30*(i+1)))*1e-6; #width of the second inductor
    ar2=w2/t2; #aspect ratio of the second inductor 
    l2= 18e-6 #length of the second inductor 
    Ic2= Jc*t2*w2 #critical current of the second inductor
    Lm2= geoindfun(l2,t2,w2,lam=lambda_NbTiN) #Calculating magnetic inductance for the second inductor
    Lk2= kinindfun(l2,t2,w2,lam=lambda_NbTiN) #Calculating kinetic inductance for the second inductor

    Ls2=Lk2*w2/l2 #sheet inductance for the second inductor 

    ratio1= Lm2/Lm1 

    #Considering the effect of both magnetic and kinetic inductance [k+m] gives      
    I1=np.linspace(0,0.999,200) #I1/Ic1
    L1=np.power(np.interp(I1, x, y),-1)*Lk1+Lm1
    L2=Lk2*np.ones(np.shape(L1))+Lm2
    I2=I1*L1/L2
    I=I1+I2
    L=np.power(np.power(L1,-1)+np.power(L2,-1),-1)
    
    fig0=plt.figure(0)
    c = next(color)        
    plt.plot(I,L1*1e12,linestyle='-',c=c)
    plt.plot(I,L2*1e12,linestyle='-',c=c)
    plt.plot(I,L*1e12,linestyle='--',label='$r= $ %d' % (ratio1),c=c )
    plt.xlim(0,4)
    plt.legend(loc ="upper right");        
    plt.xlabel('$I_{tot}/I_{c1}$');
    plt.ylabel('$L(J)$ [pH]');
    
    
    fig1=plt.figure(1)
    plt.plot(I,I1,linestyle='--',c=c)
    plt.plot(I,I2,linestyle='dashdot',label='$r= $ %d' % (ratio1),c=c)
    plt.legend(loc ="lower right")
    plt.xlim(0,3)
    plt.ylim(0,2)
    plt.xlabel('$I_{tot}/I_{c1}$')
    plt.ylabel('$I_i/I_{c1}$')
    
    print('Ic2(%d'%ratio1,')=', Ic2 )

#plt.grid();
# fig2=plt.figure(0) 
# plt.show(block=False)
# plt.grid();
# plt.figure(num=0, figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')
# plt.figure(num=1, figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')

# plt.show()
# print (Ic1)

Ic2(2 )= 0.02


In [134]:


z=np.linspace(0,0.99999,num)
Ltun= 1/np.interp(z, x, y)


#Nb
t_top= 300e-9 #thickness of the top conductor 
w_top=0.1e-6; #width of the first inductor 
ar_top=w_top/t_top; #aspect ratio of the first inductor 
lam_top=lambda_Nb
l= 312.3e-6 #length of the first inductor 
Ic_top= Jc_Nb*t_top*w_top #critical current of the first inductor
Lm_top= geoindfun(l,t_top,w_top,lam_top) #Calculating magnetic inductance for the first inductor
Lk_top= kinindfun(l,t_top,w_top,lam_top) #Calculating kinetic inductance for the first inductor
Ls_top=Lk_top*w_top/l #sheet inductance for the first inductor

# #NbTiN
# t_bottom= t1#thickness of the bottom inductor 
# w_bottom=w1; #width of the  inductor 
# ar_bottom=w_bottom/t_bottom; #aspect ratio of the  inductor 
# lam_bottom=lambda_NbTiN
# l= 312.3e-6 #length of the first inductor 
# Ic_bottom= Jc_NbTiN*t_bottom*w_bottom #critical current of the  inductor
# Lm_bottom= geoindfun(l,t_bottom,w_bottom,lam_bottom) #Calculating magnetic inductance for the  inductor
# Lk_bottom= kinindfun(l,t_bottom,w_bottom,lam_bottom) #Calculating kinetic inductance for the  inductor
# Ls_bottom=Lk_bottom*w_bottom/l #sheet inductance for the  inductor

# #Assuming:

# d=30e-9 #d is the spacing between the top and bottom conductors
# K=2 #Fringing Factor

# # print(Ltot)


# # ratio1= Lm2/Lm1 
# # ratio2= Lk2/Lk1

# ## Main inductor #1 Shunt Inductor #2 
# I1=np.linspace(0,0.999,num) #I1/Ic1
# L1=np.power(np.interp(I1, x, y),-1)*Lk_bottom+Lm_bottom
# L2=Lk2*np.ones(np.shape(L1))+Lm2
# I2=I1*L1/L2
# I=I1+I2
# L=np.power(np.power(L1,-1)+np.power(L2,-1),-1)

d=30e-9
L_bottom_tuned=L1
# L2=np.power(np.interp(I, x, y),-1)*Lk2+Lm2
L_top=(Lm_top+Lk_top)*np.ones(num,dtype='float')
Lint=np.array(np.add(L_top,L_bottom_tuned))
# print(type(Lint))
# print(len(Lint))
Ltot=Lint+l*mu0*d/w1
# plt.plot(I,Ltot*1e12)
plt.plot(I,Ltot*1e9,linestyle='-')


plt.plot(I,L_bottom_tuned*1e9,linestyle='-',c=c)
plt.plot(I,L2*1e9,linestyle='-',c=c)
plt.plot(I,L*1e9,linestyle='--',label='$r= $ %d' % (ratio1),c=c )
# plt.xlim(0,4)
plt.legend(loc ="upper right");        
plt.xlabel('$I_{tot}/I_{c1}$');
plt.ylabel('$L(J)$ [pH]');
    
# print ('Lm1=', Lm1,'Lk1=', Lk1,', Ls1=',Ls1, ', Ic1=',Ic1, 'A')
# print ('Lm2=', Lm2,'Lk2=', Lk2,', Ls2=',Ls2, ', Ic2=',Ic2, 'A')
# print ('ratio_Lm=', ratio1, ',ratio_Lk=', ratio2 )
# print ('Aspect Ratio #1=', ar1, ', Aspect Ratio #2=', ar2 )

In [17]:
#Fringing Factor K

w = []
x = []
y = []
    
with open('Fringe.csv', newline='') as f:
    reader = csv.reader(f, delimiter=',')
    
    for row in reader:
            w.extend(row)
    
           
for i in range(len(w)):
    if i%2==1:
        y.append(float(w[i]))
    else:
        x.append(float(w[i]))

# WtoD=5
# K= np.interp(WtoD, x, y)

K=interpolate.interp1d(x, y, fill_value='extrapolate')
print(K(3))
# print(y)
# plt.plot(x,y)
print(K)        


3.505479452054806


In [24]:
print('L=',(((1.2e-6*30e-9)/(3.5*100e-9))*(1+(80/30)*mpmath.coth(300/80)+(460/30)*mpmath.coth(30/460)))*1e6,'uH')

L= 24.5945795705602 uH
